In [1]:
import time
import numpy as np
from datetime import datetime
from calendar import timegm
from pymongo import MongoClient, ASCENDING, DESCENDING, ReturnDocument
import pyvisa
import serial
from pprint import pprint

## Connect to Database

In [2]:
def dbConnect():
    client = MongoClient('mongodb://localhost/')
    db = client.data
    collection = db.adr2datas
    db.collection_names()
    infocollection = db.adr2controls
    jobcollection = client.jobs.adr2jobs
    return collection, infocollection, jobcollection
collection,infocollection, jobcollection = dbConnect()

##Connect to Instruments

In [3]:
def connectInstruments():
    rm = pyvisa.ResourceManager()
    rm.list_resources()
    inst1 = rm.open_resource('GPIB0::1::INSTR', read_termination = '\x00')
    inst2 = rm.open_resource('GPIB0::2::INSTR', read_termination = '\x00')
    inst3 = rm.open_resource('GPIB0::6::INSTR', read_termination = '\x00')
    return [inst1,inst2,inst3]

##Create Information and Data Entries

In [8]:
def get_info():
    post = {
        "timeStamp": timegm(datetime.now().timetuple()),
        "fridgeStatus": "Warm",
        "currentJob": "None",
        "switchState": "Closed",
        "currentLimit": 9.2,
        "maxVoltage": 400,
        "voltageStep": 1,
        "command": "None"
    }
    return post

In [9]:
infocollection.insert(get_info())

ObjectId('56054da46545ebc411b4a4da')

In [65]:
infocollection.find_one_and_update({},{'$set': {'maxVoltage': 450}})

{u'_id': ObjectId('55f5979f6545eb56047e1f81'),
 u'command': u'Soak',
 u'currentJob': u'Soak',
 u'currentLimit': 9.2,
 u'fridgeStatus': u'Warm',
 u'maxVoltage': 450,
 u'switchState': u'Closed',
 u'timeStamp': 1442144095,
 u'voltageStep': 1}

In [4]:
def get_data(inst):
    post = {
        "timeStamp": timegm(datetime.now().timetuple()),
        "baseTemp": float(inst[1].ask('KRDG? B')),
        "oneKTemp": float(inst[0].ask('KRDG? C')),
        "threeKTemp": float(inst[0].ask('KRDG? A')),
        "sixtyKTemp": float(inst[0].ask('KRDG? B')),
        "magnetVoltage": float(inst[0].ask('SRDG? D')),
        "psVoltage": float(inst[2].ask('MEAS:VOLT?')),
        "psCurrent": float(inst[2].ask('MEAS:CURR?')),
        "currentJob": "None",
        "percentComplete": round(float(inst[2].ask('MEAS:CURR?'))/9 * 100),
        "switchState": "Closed"
    }
    return post
get_data(connectInstruments())

C:\Anaconda\lib\site-packages\pyvisa\resources\messagebased.py:387: UserWarning: read string doesn't end with termination characters
  return self.read()


{'baseTemp': 0.0,
 'currentJob': 'None',
 'magnetVoltage': -0.00051,
 'oneKTemp': 216.292,
 'percentComplete': -0.0,
 'psCurrent': -0.0319834,
 'psVoltage': -0.00941257,
 'sixtyKTemp': 112.473,
 'switchState': 'Closed',
 'threeKTemp': 207.033,
 'timeStamp': 1443214288}

## Setup Log File

In [5]:
logFileName = 'ADR2Log.txt'
inst = connectInstruments()
ps = inst[2]

In [13]:
# Open Log File
logFile = open(logFileName, 'a')

# Read Data
try:
    data = get_data(inst)

# Log errors if they occur and reconnect
except UnicodeDecodeError:
    string = "Read error at " + datetime.now().strftime('%H:%M:%S') + '\n'
    print(string)
    logFile.write(string)
    inst = connectInstruments()

# Insert Data into Database
else:
    if (data['sixtyKTemp'] <= 300):
        try:
            collection.insert(data)
        except:
            print("Database error at " + datetime.now().strftime('%H:%M:%S'))


# Get information from Database

info = infocollection.find().next()
# print('Fridge Status:')
# pprint(info)

# Control

# Change Fridge Status

# Change Job
print info['command']
if info['command'] == 'Magup':
    if info['currentJob'] != 'Magup':
        info = infocollection.find_one_and_update({},{'$set': {'currentJob': 'Magup'}})
        logString = "Start Magup at " + datetime.now().strftime('%H:%M:%S')
        print(logString)
        logFile.writelines([logString, '\n'])

elif info['command'] == 'Magdown':
    if info['currentJob'] != 'Magdown':
        info = infocollection.find_one_and_update({},{'$set': {'currentJob': 'Magdown'}})
        logString = "Start Magdown at " + datetime.now().strftime('%H:%M:%S')
        print(logString)
        logFile.writelines([logString, '\n'])

elif info['command'] == 'Soak':
    if info['currentJob'] != 'Soak':
        info = infocollection.find_one_and_update({},{'$set': {'currentJob': 'Soak'}})
        logString = "Start Soak at " + datetime.now().strftime('%H:%M:%S')
        print(logString)
        logFile.writelines([logString, '\n'])
        
else:
    print('No Command')

# Control Magnet
job = info['currentJob']

if job == 'Magup':
    # Increase Magnet Voltage
    currentV = float(ps.ask('VOLT?'))*1000
    if currentV < info['maxVoltage']:
        nextV = currentV + info['voltageStep']
#         ps.write('VOLT:LEV:IMM ' + str(nextV) + 'mV')
        print(str(nextV) + "mV\n" + "Current: " + ps.ask('MEAS:CURR?') + "Voltage: " + ps.ask('MEAS:VOLT?'))
    else:
        print('Max Voltage Reached')
        info = infocollection.find_one_and_update({},{'$set': {'command': 'Soak'}})
elif job == 'Magdown':
    # Decrease Magnet Voltage
    currentV = float(ps.ask('VOLT?'))*1000
    if currentV > 0:
        nextV = currentV - info['voltageStep']
#         ps.write('VOLT:LEV:IMM ' + str(nextV) + 'mV')
        print(str(nextV) + "mV\n" + "Current: " + ps.ask('MEAS:CURR?') + "Voltage: " + ps.ask('MEAS:VOLT?'))
    else:
        print('Zero Voltage Reached')
        info = infocollection.find_one_and_update({},{'$set': {'command': 'None'}})
elif job == 'Soak':
    # Soak
    print('soak')
else:
    print('No Job')

# Close Log File
logFile.close()

# Wait
# time.sleep(5)

Magdown
Start Magdown at 09:41:11
1.0mV
Current: -3.74880E-2Voltage: -9.41257E-3


In [14]:
# Check Jobs

# Find next job
nextJob = jobcollection.find({}).limit(1).sort('startTime', ASCENDING):
print("Next Job at " + nextJob['timeString'])

True

In [ ]:
currentJob = None
while True:
    # Open Log File
    logFile = open(logFileName, 'a')

    # Read Data
    try:
        data = get_data(inst)

    # Log errors if they occur and reconnect
    except UnicodeDecodeError:
        string = "Read error at " + datetime.now().strftime('%H:%M:%S') + '\n'
        print(string)
        logFile.write(string)
        inst = connectInstruments()

    # Insert Data into Database
    else:
        if (data['sixtyKTemp'] <= 300):
            try:
                collection.insert(data)
            except:
                print("Database error at " + datetime.now().strftime('%H:%M:%S'))

    # Check for impending jobs
#     nextJob = None
#     for job in jobcollection.find({}).limit(1).sort('startTime', ASCENDING):
#         nextJob = job
#     if (nextJob):
#         print("Next Job at " + nextJob['timeString'])
    
#     if (nextJob and (nextJob['startTime'] - datetime.utcnow()).seconds < 5):
#         # Mark previous current job as completed
#         if (currentJob and currentJob['inProgress']):
#             jobcollection.find_one_and_update({'_id': currentJob['_id']}, {'$set': {'inProgress': False, 'completed': True}})
        
#         # Next Job becomes current job
#         currentJob = nextJob
#         print("Starting Scheduled " + currentJob['type'])
#         jobcollection.find_one_and_update({'_id': currentJob['_id']}, {'$set': {'inProgress': True}})
# #         info = infocollection.find_one_and_update({},{'$set': {'command': currentJob['type']}})
    
    # Get information from Database

    info = infocollection.find().next()

    if info['command'] == 'Magup':
        #Ensure fridge is actually cold enough to mag up!
        if data['threeKTemp'] > 3.5:
            pass
        elif info['currentJob'] != 'Magup':
            info = infocollection.find_one_and_update({},{'$set': {'currentJob': 'Magup'}})
            logString = "Start Magup to " + str(info['maxVoltage']) + "mV at " + datetime.now().strftime('%a, %c')
            print(logString)
            logFile.writelines([logString, '\n'])

    elif info['command'] == 'Magdown':
        if info['currentJob'] != 'Magdown':
            if info['currentJob'] == 'Soak':
                stopSoakTime = datetime.now()
                t = (stopSoakTime - startSoakTime)
                logString = "Soaked for {} days, {}h: {}m: {}s".format(t.days,t.seconds//3600,(t.seconds//60)%60, t.seconds%60)
                print(logString)
                logFile.writelines([logString, '\n'])
            
            info = infocollection.find_one_and_update({},{'$set': {'currentJob': 'Magdown'}})
            logString = "Start Magdown at " + datetime.now().strftime('%a, %c')
            print(logString)
            logFile.writelines([logString, '\n'])

    elif info['command'] == 'Soak':
        if info['currentJob'] != 'Soak':
            info = infocollection.find_one_and_update({},{'$set': {'currentJob': 'Soak'}})
            logString = "Start Soak at " + datetime.now().strftime('%a, %c')
            startSoakTime = datetime.now()
            print(logString)
            logFile.writelines([logString, '\n'])

    else:
        info = infocollection.find_one_and_update({},{'$set': {'currentJob': 'None'}})
        if data['baseTemp'] < 3:
            statusString = "Cold (Magged Down)"
        elif data['threeKTemp'] < 3.5:
            statusString = "Cold (3K)"
        elif (data['sixtyKTemp'] < data['threeKTemp']) and (data['sixtyKTemp'] > 70):
            statusString = "Cooling Down"
        elif (data['sixtyKTemp'] > data['threeKTemp']) and (data['sixtyKTemp'] > 70):
            statusString = "Warming Up"
        else:
            statusString = "Warm"
        
    

    # Control Magnet
    job = info['currentJob']

    if job == 'Magup':
        # Increase Magnet Voltage
        currentV = float(ps.ask('VOLT?'))*1000
        if currentV < info['maxVoltage']:
            nextV = currentV + info['voltageStep']
            ps.write('VOLT:LEV:IMM ' + str(nextV) + 'mV')
            print(str(nextV) + "mV\n" + "Current: " + ps.ask('MEAS:CURR?') + "\tVoltage: " + ps.ask('MEAS:VOLT?'))
        else:
            print('Max Voltage Reached')
            if data['psCurrent'] > 9.0:
                print('9A reached')
                info = infocollection.find_one_and_update({},{'$set': {'command': 'Soak'}})
        statusString = "Magup"
    elif job == 'Magdown':
        # Decrease Magnet Voltage
        currentV = float(ps.ask('VOLT?'))*1000
        if currentV > 0:
            nextV = currentV - info['voltageStep']
            ps.write('VOLT:LEV:IMM ' + str(nextV) + 'mV')
            print(str(nextV) + "mV\n" + "Current: " + ps.ask('MEAS:CURR?') + "\tVoltage: " + ps.ask('MEAS:VOLT?'))
        else:
            print('Zero Voltage Reached')
            info = infocollection.find_one_and_update({},{'$set': {'command': 'None'}})
        statusString = "Magdown"
    elif job == 'Soak':
        # Soak
        statusString = "Soak"
    else:
        pass
    
    info = infocollection.find_one_and_update({},{'$set': {'fridgeStatus': statusString}})
    # Close Log File
    logFile.close()

    # Wait
    time.sleep(5)

In [30]:
logstring

'Soaked for 0 days, 0h: 0m: 10s'